# AI in Games, _Reinforcement Learning_<br>Assignment 2, Question 4:<br>**Non-tabular Model-free Methods**

## Preparing the context
The following are the necessary preparations and imports needed to run and test the main code of this document in the intended context. Mounting directory & setting present working directory...

In [ ]:
if __name__ == '__main__':
    # Mounting the Google Drive folder (run if necessary):
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    # Saving the present working directory's path:
    # NOTE: Change `pwd` based on your own Google Drive organisation
    pwd = "./drive/MyDrive/ColabNotebooks/AIG-Labs/AIG-Assignment2/"

Mounted at /content/drive/


To install module `import_ipynb` to enable importing Jupyter Notebooks as modules...

`!pip install import_ipynb`

Importing the code in notebook `Q1_environment.ipynb`...




In [ ]:
if __name__ == '__main__':
    import import_ipynb
    N = import_ipynb.NotebookLoader(path=[pwd])
    N.load_module("Q1_environment")
    from Q1_environment import *

importing Jupyter notebook from ./drive/MyDrive/ColabNotebooks/AIG-Labs/AIG-Assignment2/Q1_environment.ipynb


**NOTE**: `Q1_environment` contains

- The environment class and `FrozenLake` subclass
- The lists containing the small and big lake environments
- The function for rendering the policy and state-values
- The function for implementing the epsilon-greedy policy

Other necessary imports...

In [ ]:
import numpy as np

## $\epsilon$-greedy policy
The $\epsilon$-greedy (epsilon-greedy) policy is a method of choosing an action from a state such that the probability of choosing a random action is $\epsilon$ while the probability of choosing the action that maximises the action-value function (as far as it has been estimated) from the given state is $1-\epsilon$. $\epsilon$ (epsilon) is called the "exploration factor". $\epsilon$-greedy is a solution to the exploration-exploitation dilemma, wherein the degree of exploration (i.e. expansion of actions unknown potential) is decided by the exploration factor.

<br>**NOTE**: **Breaking ties between reward-maximising actions**:
<br>When the above policy chooses to exploit rather than explore, it may be the case that there exist multiple actions that (within a tolerance level) maximise the action-value function from a given state. In such a case, exploration can be encouraged even during exploitation by making a random selection from the reward-maximising actions. This approach is _no worse and potentially better_ than simply picking the first action that maximises the action-value function from the given state, since it furthers exploration without reducing exploitation.

In [ ]:
# NOTE: The following is generalised for the tabular & non-tabular methods:
def e_greedy(q, e, n_actions, random_state, s=None):
    '''
    NOTE ON THE ARGUMENTS:
    - `q`: One of the following...
        - The matrix of action-values for each state
        - The array of action-values for a given state
    - `e`: The exploration factor epsilon
    - `n_actions`: The number of actions an agent can take from any state
    - `random_state`: The set `numpy.random.RandomState` object
    - `s`: The given state

    If `s=None`, `q` is the array of action-values for a given state. Else,
    `q` is the matrix of action-values for each state.
    '''

    # Storing the action-values for the given state `s` (if `s` is given):
    if s != None: q = q[s]

    # The epsilon-greedy policy:
    if random_state.rand() < e:
        return random_state.randint(0,n_actions)
    else:
        # Obtaining all actions that maximise action-value from state `s`:
        best = [a for a in range(n_actions) if np.allclose(np.max(q), q[a])]
        # Breaking the tie using random selection:
        return random_state.choice(best)

## Wrapping the environment to enable feature mapping
The class `LinearWrapper` implements a wrapper that forms a layer of abstraction around an environment object that is given to its constructor, and the wrapper class has similar functionalities as this environment object's class. However, the methods reset and step return a feature matrix when they would typically return
a state `s`.
<br><br>**Structure of the feature matrix**...
<br>In general, a feature map $\phi$ is a function that maps each state $s \in S$ to an $m$ dimensional feature vector, i.e. $\phi: S \rightarrow \mathbb{R}^m$. Hence, in general, a feature matrix can be thought of as a collection of feature vectors wherein each feature vector maps to a particular state. In other words, a feature matrix is a matrix wherein each row corresponds to a state, and each column corresponds to a particular feature; each features vector helps describe a state, and the feature matrix helps describe the collection of all states.
<br><br>However, in our case, since the agent needs to learn the action-value function (which is based on state-action pairs), the feature map $\phi$ is a function that maps each state-action pair $(s, a) \in S \times A$ to an $m$ dimensional feature vector, i.e. $\phi: S \times A \rightarrow \mathbb{R}^m$. Hence, here, the feature matrix can be thought of as a collection of feature vectors wherein each feature vector maps to a particular state-action pair. How exactly such feature mapping is implemented is discussed after the following code block.

In [ ]:
class LinearWrapper:
    def __init__(self, env):
        self.env = env
        self.n_actions = self.env.n_actions
        self.n_states = self.env.n_states
        self.n_features = self.n_states * self.n_actions

    #================================================

    # Mapping the given state paired with each action to vectors of features:

    def encode_state(self, s):
        # Initialising the feature matrix:
        features = np.zeros((self.n_actions, self.n_features))
        for a in range(self.n_actions):
            i = np.ravel_multi_index((s, a), (self.n_states, self.n_actions))
            # Updating the feature matrix:
            features[a, i] = 1.0
        '''
        EXPECTED RESULT:
        `features` is such that each row i corresponds to an action i, each
        column corresponds to a state-action pair (see implementation notes
        for more clarity on the structure), and for each row i, 1.0 is assigned
        only to those indices that correspond the given state s and action i.
        Hence, each row vector is 1.0 at only one position and 0 in all others.
        '''

        return features

    #================================================

    # Obtaining the policy via decoding the feature matrix:

    def decode_policy(self, theta):
        # Initialising the policy & state value arrays:
        policy = np.zeros(self.env.n_states, dtype=int)
        value = np.zeros(self.env.n_states)
        #------------------------------------
        # Decoding the action-value function & obtaining policy & state values:
        for s in range(self.n_states):
            features = self.encode_state(s)
            q = features.dot(theta)
            policy[s] = np.argmax(q)
            value[s] = np.max(q)
            '''
            NOTE ON THE NATURE OF `q`:
            `q` is calculated each time w.r.t. the given state `s`; do not
            consider it as an array mapping every possible state-action pair to
            a value, but rather an array that, given a state, maps every action
            to a value (effectively mapping the state-action values for the
            particular state)
            '''
        #------------------------------------
        # Returning obtained policy & state values:
        return policy, value

    #================================================

    # Resetting environment & encoding it as feature vector:

    def reset(self):
        return self.encode_state(self.env.reset())

    #================================================

    # Taking a step in environment & encoding next state as feature vector:

    def step(self, action):
        state, reward, done = self.env.step(action)
        return self.encode_state(state), reward, done

    #================================================

    # Visualising the agent's performance (by inputs or using a policy):

    def render(self, policy=None, value=None):
        self.env.render(policy, value)

**PROGRAMMING NOTE**: `numpy.ravel_multi_index` **as used in the code**:<br>Consider the following:

- $A$ is an $m \times n$ array
- $R$ is a vector containing certain row indices of $A$
- $C$ is a vector containing certain column indices of $A$
- $(R, C)$ together refer to particular cells of $A$

In other terms, $R$ is vector containing indices referring to particular rows of $A$, while $C$ is a vector containing indices referring to particular columns of $A$. When put together as a tuple $(R, C)$, we have that $(R[i], C[i])$ refers to a particular element of $A$ for each $i$. Hence, $R$ and $C$ have to be of equal size; if either one is an integer, we consider it the same way we would consider a vector of equal size as the other with each element being this integer.<br>**EXAMPLE**: $([1, 2, 3], 0) \equiv ([1, 2, 3], [0, 0, 0])$
<br>**KEY NOTE**: If both $R$ and $C$ are integers, then $(R, C)$ refers to a single element.
<br><br>`numpy.ravel_multi_index` accepts the following main arguments:

- `multi_index` = $(V_1, V_2 ... V_k)$
- `dims` = $(m_1, m_2 ... m_k)$

In general, given a $k$-dimensional array $A$ with dimensionality $m_1 \times m_2 \times ... \times m_k$, `multi_index` refers to the multi-dimensional indices wherein the $i$-th element $V_i$ of the tuple is the index or vector of indices for the $i$-th dimension of $A$. The function `numpy.ravel_multi_index` returns the indices for the same element(s) of $A$ considering a flattened (1-dimensional) version of $A$.
<br><br>When referring to a 2-dimensional $m \times n$ array:

- `multi_index` = $(R, C)$
- `dims` = $(m, n)$

In our case in particular, the arguments become:

- `multi_index` = `(s, a)` (`s` and `a` are integers)
- `dims` = `(self.n_states, self.n_actions)`

Here, `s` is the integer referring a particular state and `a` is the integer referring to a particular action. Hence, the index is $(s, a)$, and we are considering a hypothetical `n_states` $\times$ `n_actions` array $A$ to which this index refers to. What we obtain is the corresponding index for the flattened version of this hypothetical array $A$.
<br><br>**SIDE NOTE**: **Optional argument** `order`:<br>This argument determines whether the multi-index should be viewed as indexing in row-major (C-style) or column-major (Fortran-style) order. The former is the default and can be specified as `'C'`, while the latter is specified `'F'`. To make it clearer, row-major assumes that the array that the indices refer to is flattened in a left-to-right order starting from the first (top-most) row, whereas column-major assumes that this array is flattened in a top-to-bottom order starting from the first (left-most) column.

---

**IMPLEMENTATION NOTE**: **Feature mapping method**:<br>The feature matrix $M_{\phi}$ is an $|A| \times (|S| \cdot |A|)$ array, where $A$ is the collection of all actions (with cardinality $|A|$) and $S$ is the collection of all states (with cardinality $|S|$). We consider each row vector as a flattened $|S| \times |A|$ array (_flattened in row-major order by default_) that pairs each state $i$ (corresponding to row $i$) to each action $j$ (corresponding to column $j$). For convenience, let us call this row vector $V_{(s,a)}$, which is duplicated for each action in the feature matrix $M_{\phi}$.
<br><br>Now, considering the feature matrix $M_{\phi}$, we initialise this matrix as follows. For each row $i$ of $M_{\phi}$ (which corresponds to action $i$), we assign $1$ to all those indices of the $i$-th duplicate of $V_{(s,a)}$ that refer to any state paired with action $i$; we assign $0$ for all other indices. Hence, though each action is technically mapped to $|S| \cdot |A|$ features, in practice, each state-action pair is mapped to one feature. In fact, practically, the whole feature matrix $M_{\phi}$ is the feature map with respect to one state and all actions. Furthermore, $Q$, i.e. the action reward function (represented as an array), is updated such that $Q(a)$ returns the action reward for $a$ given a predefined state $s$.

<br>**Reasons for this representation**...
<br>The above explained representation has the following advantages:

- Action value array needs to be only as large as `n_actions`, since it is obtained anew given a state
- Getting dot product between parameter vector $\theta$ and each feature vector is made easy, since:
  - Each feature vector is of the same size (`n_states` $\times$ `n_actions`) as $\theta$
  - Each feature vector's elements correspond to those of $\theta$
  - Irrelevant positions in feature vector for given action-state pair are $0$, hence do not contribute

**NOTE**: The dot products between the parameter vector $\theta$ and the feature vectors are used to obtain the action reward values $Q$ (for given a state), and are also used to update $\theta$ for the next iteration.

## SARSA with linear action value function approximation

In [ ]:
def linear_sarsa(wenv, max_episodes, eta, gamma, epsilon, seed=None):
    '''
    NOTE ON THE ARGUMENTS:
    - `wenv`:
        - Wrapped object of a chosen environment model (ex. FrozenLake)
        - Contains mechanisms to map each state to a feature vector
        - Helpful in estimating action-value function as a linear function
        - Also helps decode feature vectors to estimated optimal policies
    - `max_episodes`: Upper limit of episodes the agent can go through
    - `eta`:
        - Initial learning rate
        - The learning rate is meant to decrease linearly over episodes
    - `gamma`:
        - Discount factor
        - Not subject to change over time
    - `epsilon`:
        - Initial exploration factor
        - Exploration factor is w.r.t. epsilon-greedy policy
        - Denotes the chance of selecting a random state
        - The exploration factor is meant to decrease linearly over episodes
    - `seed`:
        - Optional seed for pseudorandom number generation
        - By default, it is `None` ==> random seed will be chosen
    '''
    # INITIALISATION
    # Setting random state with `seed` for enabling replicability:
    random_state = np.random.RandomState(seed)

    # Initialising key parameters:
    eta = np.linspace(eta, 0, max_episodes)
    epsilon = np.linspace(epsilon, 0, max_episodes)
    theta = np.zeros(wenv.n_features)

    #================================================

    # LEARNING LOOP
    for i in range(max_episodes):
        # NOTE: i ==> episode number
        # Beginning at the initial state before each episode:
        features = wenv.reset()
        # NOTE: `features` here represents the initial state
        q = features.dot(theta)
        # NOTE: `q` here is the rewards per action for the initial state
        '''
        NOTE ON THE SHAPE OF `q`:
        `features.dot(theta)` produces `wenv.n_actions` dot products by
        applying dot product between `theta` and each row vector of the matrix
        `features`.
        '''
        a = e_greedy(q, epsilon[i], wenv.n_actions, random_state)

        done = False
        while not done:
            next_features, r, done = wenv.step(a)
            # NOTE: `next_features` here represents the next state reached

            # Obtaining part of the temporal difference of `(features, a)`:
            delta = r - q[a]

            # Selecting action `a` for `next_features`:
            # NOTE: Selection is done by epsilon greedy policy based on `q`
            q = next_features.dot(theta)
            # NOTE: `q` here is the rewards per action for the next state
            next_a = e_greedy(q, epsilon[i], wenv.n_actions, random_state)
            # NOTE: `next_a` is the action taken in the next state

            # Obtaining the full temporal difference of `(features, a)`:
            delta += gamma*q[next_a]

            # Updating model parameters `theta`:
            theta += eta[i]*delta*features[a]
            # `next_features[a]` is feature vector for state `s` & action `a`

            # Moving to the next state & its corresponding action:
            features, a = next_features, next_a

    # Returning the parameter vector `theta`:
    return theta

## Q-Learning with linear action value function approximation

In [ ]:
def linear_q_learning(wenv, max_episodes, eta, gamma, epsilon, seed=None):
    '''
    NOTE ON THE ARGUMENTS:
    Same as for the function `linear_sarsa`.
    '''
    # Setting random state with `seed` for enabling replicability:
    random_state = np.random.RandomState(seed)

    # Initialising key parameters:
    eta = np.linspace(eta, 0, max_episodes)
    epsilon = np.linspace(epsilon, 0, max_episodes)
    theta = np.zeros(wenv.n_features)

    #================================================

    # LEARNING LOOP
    for i in range(max_episodes):
        # NOTE: i ==> episode number
        # Beginning at the initial state before each episode:
        features = wenv.reset()
        # NOTE: `features` here represents the initial state
        q = features.dot(theta)
        # NOTE: `q` here is the rewards per action for the initial state
        '''
        NOTE ON THE SHAPE OF `q`:
        See the corresponding comment for the function `linear_sarsa`.
        '''
        a = e_greedy(q, epsilon[i], wenv.n_actions, random_state)

        done = False
        while not done:
            next_features, r, done = wenv.step(a)
            # NOTE: `next_features` here represents the next state reached

            # Obtaining part of the temporal difference of `(features, a)`:
            delta = r - q[a]

            # Selecting action `a` for `next_features`:
            # NOTE: Selection is done by epsilon greedy policy based on `q`
            q = next_features.dot(theta)
            # NOTE: `q` here is the rewards per action for the next state
            max_a = np.argmax(q)
            # NOTE: `max_a` is the action maximising `q` for next state

            # Obtaining the full temporal difference of `(features, a)`:
            delta += gamma*q[max_a]

            # Updating model parameters `theta`:
            theta += eta[i]*delta*features[a]
            # `next_features[a]` is feature vector for state `s` & action `a`

            # Moving to the next state & its corresponding action:
            features, a = next_features, e_greedy(q, epsilon[i], wenv.n_actions, random_state)

    # Returning the parameter vector `theta`:
    return theta

## Testing the above functions
_The function testing code must not run if this file is imported as a module, hence we do..._<br>`if __name__ == '__main__'`<br>_... to check if the current file is being executed as the main code._

In [ ]:
if __name__ == '__main__':
    # Defining the parameters:
    env = FrozenLake(lake=LAKE['small'], slip=0.1, max_steps=None, seed=0)
    # NOTE: Putting `max_steps=None` makes it default to the grid size
    wenv = LinearWrapper(env)
    max_episodes = 2000
    eta = 1
    gamma = 0.9
    epsilon = 1

    # Running the functions:
    _LSARSA = linear_sarsa(wenv, max_episodes, eta, gamma, epsilon, 0)
    _LQLearning = linear_q_learning(wenv, max_episodes, eta, gamma, epsilon, 0)

    LSARSA = wenv.decode_policy(_LSARSA)
    LQLearning = wenv.decode_policy(_LQLearning)
    labels = ("linear sarsa", "linear q-learning")

    # Displaying results:
    displayResults((LSARSA, LQLearning), labels, env)



AGENT PERFORMANCE AFTER LINEAR SARSA

Lake:
[['&' '.' '.' '.']
 ['.' '#' '.' '#']
 ['.' '.' '.' '#']
 ['#' '.' '.' '$']]
Policy:
[['_' '>' '_' '<']
 ['_' '^' '_' '^']
 ['>' '>' '_' '^']
 ['^' '>' '>' '_']]
Value:
[[0.365 0.413 0.494 0.346]
 [0.435 0.000 0.596 0.000]
 [0.542 0.638 0.761 0.000]
 [0.000 0.776 0.890 1.000]]


AGENT PERFORMANCE AFTER LINEAR Q-LEARNING

Lake:
[['&' '.' '.' '.']
 ['.' '#' '.' '#']
 ['.' '.' '.' '#']
 ['#' '.' '.' '$']]
Policy:
[['_' '<' '_' '<']
 ['_' '^' '_' '^']
 ['>' '>' '_' '^']
 ['^' '>' '>' '_']]
Value:
[[0.427 0.386 0.369 0.359]
 [0.482 0.000 0.541 0.000]
 [0.573 0.685 0.784 0.000]
 [0.000 0.739 0.891 1.000]]
